Data Mining and Visualisation 2019-2020 <br>
Practical 5 - Time-Series Analysis and Processing - focus on Time-Series Classification<br>
Teaching Assistant Muhammad Usman <br>

This practical requires you to use time series classification approaches to classify multivariate sensor data based on whether the telecommunications network is actually up and running or not. -1 is the negative class denoting downtime and 1 is for optimal network condition. <br>
Use this notebook with CoLab. <br>
You may enable GPU capabilities <br>

You need to undergo a number of steps to fill the gaps below.

In [0]:
# Load all necessary libaries, including keras and tensorflow. 
import keras
import tensorflow as tf
import os
import pandas as pd
import numpy as np

# Load data



In [4]:
from google.colab import files
files.upload() #upload the files needed with this command if you'd like. There are about 20 csv files and one for the target labels

{}

In [5]:
!ls

sample_data		 Wireless_Telecom_17.csv  Wireless_Telecom_3.csv
Wireless_Telecom_10.csv  Wireless_Telecom_18.csv  Wireless_Telecom_4.csv
Wireless_Telecom_11.csv  Wireless_Telecom_19.csv  Wireless_Telecom_5.csv
Wireless_Telecom_12.csv  Wireless_Telecom_1.csv   Wireless_Telecom_6.csv
Wireless_Telecom_13.csv  Wireless_Telecom_20.csv  Wireless_Telecom_7.csv
Wireless_Telecom_14.csv  Wireless_Telecom_21.csv  Wireless_Telecom_8.csv
Wireless_Telecom_15.csv  Wireless_Telecom_22.csv  Wireless_Telecom_9.csv
Wireless_Telecom_16.csv  Wireless_Telecom_2.csv   Wireless_Telecom_target.csv


In [0]:
data_files = [f for f in os.listdir() if f.endswith('csv')]

In [14]:
#Read one of the files and printi out with name.head(). Also try to return its shape with name.shape
df = pd.read_csv(data_files[0])
df.head()

,Sensor 1,Sensor 2,Sensor 3,Sensor 4
0,-1.00000,-0.64,0.28571,0.35
1,-0.61905,-0.20,0.71429,0.30
2,-0.76190,-0.32,0.42857,0.25
3,-0.85714,-0.36,0.85714,0.20
4,-0.57143,-0.64,0.85714,0.35


In [15]:
df.shape

(24, 4)

In [23]:
# Load all data files
sequences = []
path = 'Wireless_Telecom'

for i in range(1, len(data_files)):
    file_path = f'{path}_{i}.csv'
    print(file_path)
    df = pd.read_csv(file_path, header=0)
    values = df.values
    sequences.append(values)

Wireless_Telecom_1.csv
Wireless_Telecom_2.csv
Wireless_Telecom_3.csv
Wireless_Telecom_4.csv
Wireless_Telecom_5.csv
Wireless_Telecom_6.csv
Wireless_Telecom_7.csv
Wireless_Telecom_8.csv
Wireless_Telecom_9.csv
Wireless_Telecom_10.csv
Wireless_Telecom_11.csv
Wireless_Telecom_12.csv
Wireless_Telecom_13.csv
Wireless_Telecom_14.csv
Wireless_Telecom_15.csv
Wireless_Telecom_16.csv
Wireless_Telecom_17.csv
Wireless_Telecom_18.csv
Wireless_Telecom_19.csv
Wireless_Telecom_20.csv
Wireless_Telecom_21.csv
Wireless_Telecom_22.csv


In [26]:
targets = pd.read_csv(f'{path}_target.csv').values[:,1]
targets

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1])

In [27]:
# Sequences have different lengths. So we need to make that uniform based on some statistics.
# You need to provide some statistics with regards to the each sequence lengths, i.e. some sequences are longer than others.
# use .describe() to get an idea of what the spread of the sequence lengths is
len_sequences = [len(s) for s in sequences]
pd.Series(len_sequences).describe()

count    22.000000
mean     25.818182
std       2.152216
min      23.000000
25%      24.000000
50%      25.500000
75%      27.000000
max      30.000000
dtype: float64

In [29]:
print(len_sequences)

[24, 24, 24, 24, 24, 24, 23, 27, 28, 24, 26, 29, 29, 25, 30, 27, 29, 23, 25, 27, 26, 26]


In [0]:
#add padding to the max length based on the value you got from .describe()
to_pad = max(len_sequences)
new_seq = []
for one_seq in sequences:
    len_one_seq = len(one_seq)
    last_val = one_seq[-1]
    n = to_pad - len_one_seq
    to_concat = np.repeat(one_seq[-1], n).reshape(4, n).transpose()
    new_one_seq = np.concatenate([one_seq, to_concat])
    new_seq.append(new_one_seq)
current_seq = np.stack(new_seq)

In [0]:
#truncate the sequences to having 24 values

In [0]:
#now create train and test sets..... for the sake of this example we will consider the test set as being both a test and validation set, but you can just adjust it accordingly if you want to create a validation set as well
train = [current_seq[i] for i in range(1,16)]
test = [current_seq[i] for i in range(17,21)]

train_target = [targets[i] for i in range(1,16)]
test_target = [targets[i] for i in range(17,21)]

In [0]:
train = np.array(train)
test = np.array(test)

train_target = np.array(train_target)
train_target = (train_target+1)/2

test_target = np.array(test_target)
test_target = (test_target+1)/2

In [0]:
#create an LSTM model having 512 units. It is a binary problem so you can use a dense layer with value 1 and activation sigmoid
model = Sequential()
model.add####
model.add####

In [0]:
#now you need to actually define the training process for the LSTM by adding optimiser (use ADAM or any other you want) and also play with different values for learning rate
#create a check point so that you can get an outcome for each epoch
model.compile (#binary cross entropy, optimiser, metrics=['accuracy'])
model.fit(train, train_target, epochs=200, batch_size=32, callbacks=[chk], validation_data=(test,test_target))

In [0]:
#load the model created in order to estimate the accuracy on the training set
#use the model.predict_classes to get the dataset you need to make the predictions and then compare the test_predicted values against the test_target one to get the final accuracy score



##end of exercise###